Import necessary libraries

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import os
import sys

import pickle
import tensorflow as tf


import warnings
warnings.filterwarnings("ignore")
import os
from skimage import io, transform
import cv2

import editdistance
import get_feature_vector as gfv;
import get_label_vector as glv
from sklearn.model_selection import train_test_split

import numpy as np
from skimage import color
from skimage import filters
from skimage import morphology, util
from skimage import morphology as mp;
import AdjustImage as ai
from skimage import img_as_uint

LOGDIR = "model"

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


Create necessary funcitons

In [2]:
sentence = ["আমি ভাত খাই","আমি ভাত খাই","আমি ভাত খাই","তুমি কেমন আছো","তুমি কেমন আছো","তুমি কেমন আছো",
            "আমার নাম রহিম","আমার নাম রহিম","আমার নাম রহিম","পুলিশকে ডাক","পুলিশকে ডাক","পুলিশকে ডাক","বিদায় নিচ্ছি","বিদায় নিচ্ছি",
            "বিদায় নিচ্ছি","একটি ভাষা যথেষ্ট নয়","একটি ভাষা যথেষ্ট নয়","একটি ভাষা যথেষ্ট নয়","সে কোন দেশের লোক","সে কোন দেশের লোক",
            "সে কোন দেশের লোক","আপনি কি বাংলা বলতে পারেন","আপনি কি বাংলা বলতে পারেন","আপনি কি বাংলা বলতে পারেন", "মাফ করবেন",
            "মাফ করবেন", "মাফ করবেন", "আমি তোমাকে ভালবাসি","আমি তোমাকে ভালবাসি","আমি তোমাকে ভালবাসি"]
def rescale(img, sz):
    h,w = img.shape;
    scale = sz/h;
    img = np.array(img, dtype=np.uint8)
    img = cv2.resize(img,None,fx=scale,fy=scale)
    
    return img;

def resize2SquareKeepingAspectRation(img, size, interpolation):
    img = np.array(img, dtype=np.uint8)
    h, w = img.shape[:2]
    c = None if len(img.shape) < 3 else img.shape[2]
    if h == w: return cv2.resize(img, (size, size), interpolation)
    if h > w: dif = h
    else: dif = w
    x_pos = int((dif - w)/2.)
    y_pos = int((dif - h)/2.)
    if c is None:
        mask = np.ones((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        mask = np.ones((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]
    return cv2.resize(mask, (size, size), interpolation)


def toSparse(output_label):
    #put ground truth text into sparse tuple for ctc_loss
    
    indices = [];
    values = [];
    shape = [len(output_label), 0] #shape[1] must be max(labelList)
    
    #go over all batches
    for (batchElement, label) in enumerate(output_label):
        #get list of labels
        labels = [c for c in label];
        
        #sparse tensor must have size of max labels
        if(len(labels) > shape[1]):
            shape[1] = len(labels)
            
        #put each label on sparse tensor
        for(i, lbl) in enumerate(labels):
            indices.append([batchElement, i]);
            values.append(lbl);
    
    return (indices, values, shape);

myFile = 'borno.txt';
with open(myFile , 'rb') as f:
    rs_borno = pickle.load(f);

#sort the arrayz
rs_borno.sort();
#print(rs_borno);

def to_string(ar, rs_borno):
    #print(ar);
    ans = "";
    
    #ans2 = list();
    vowel_dep = ['া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', 'ং', '্'];
    char_set = list();
    
    indx = 0 ;
    character = "";
    while(indx<len(ar)):
        ch = rs_borno[ar[indx]];
        
        if(ch != '্'):
            if(len(character) > 0):
                char_set.append(character);
                character = "";
            
            if(ch not in vowel_dep):
                character += ch;
            
            else:
                char_set.append(ch);
        else:
            character += ch;
            if(indx+1 < len(ar)):
                indx+=1;
                character += rs_borno[ar[indx]];
        indx+=1;
    
    if(len(character)>0):
        char_set.append(character);
    #print(char_set);
    ind = 0;
    while(ind<len(char_set)):
        ch = char_set[ind];
        if(ch == 'ে'):
            if(ind+2<len(char_set) and char_set[ind+2] == 'া'):
                ans += char_set[ind+1];
                ans += 'ো';
                ind+=2;
            elif(ind+2<len(char_set) and char_set[ind+2] == 'ী'):
                ans += char_set[ind+1];
                ans += 'ৌ';
                ind+=2;
            else:
                if(ind+1<len(char_set)):
                    ans += char_set[ind+1];
                    ind+=1;
                ans += ch;
        elif(ch == 'ি' or ch =='ৈ'):
            if(ind+1<len(char_set)):
                ans += char_set[ind+1];
                ind+=1;
            ans += ch;
        else:
            ans += ch;
        ind+=1;
    return ans; #, ans2;

#test form http://localhost:8888/notebooks/Normal_practice/Bangla_Processing/Processing%20Bangla%20Word.ipynb
print(to_string([34, 50, 31, 54, 29, 54, 37, 44, 32, 44, 30, 54, 37, 44, 58], rs_borno));

বন্দ্যোপাধ্যায়


# Create Model object (BLSTM + CTC)

In [3]:
def lstm_cell(is_training, hidden_size, dropout):
    cell = tf.contrib.rnn.LSTMCell(num_units = hidden_size, forget_bias=1.0, name = 'LSTM');
    if is_training and dropout < 1:
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=dropout)
    return cell

class Model(object):
    #constructor
    def __init__(self, is_training, batch_size, num_steps, hidden_size, no_of_output_labels, num_layers, dropout = 0.4, init_scale = 0.05):
        self.is_training = is_training;
        
        self.batch_size = batch_size;
        self.num_steps = num_steps;
        self.hidden_size = hidden_size;
        self.num_layers = num_layers;
        self.snap_id = 0;
        
        with tf.name_scope("Input"):
            self.inputs = tf.placeholder(tf.float32, shape=(self.batch_size, self.num_steps, 9));
        
        with tf.name_scope("Dropout"):
            if(is_training and dropout < 1):
                self.inputs = tf.nn.dropout(self.inputs, dropout);
        
        #setup Long Short Term Memory Network
        
        
        with tf.name_scope("LSTM_cell"):
            #create an LSTM Cell
            cells = [lstm_cell(is_training, hidden_size, dropout) for _ in range(num_layers)] # 2 layers
            # stack basic cells
            stacked = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)
            
        with tf.name_scope("BLSTM_layer"):   
            # bidirectional RNN
            # BxTxF -> BxTx2H
            ((fw, bw), _) = tf.nn.bidirectional_dynamic_rnn(cell_fw=stacked, cell_bw=stacked, inputs=self.inputs, dtype=self.inputs.dtype)

            #concatenate the forward and backward output
            #tf.concat(data, 2) concatenates dimention no. 2
            #tf.expand_dims(data, 2) creates a new dimention at position 2 (done because of CTC ouptup layer)
            #tus BxTxH + BxTxH -> BxTx2H -> BxTx1X2H
            concat_out = tf.expand_dims(tf.concat([fw, bw], 2), 2);



            #reshape the output into BxTx1x2H -> (BxTx1)x2H -> BT1x2H to perform the affine transform
            concat_out = tf.reshape(concat_out, [-1, 2*hidden_size]);
        
        
        with tf.name_scope("Affine_Transform"):
            # Perform Affine Transform
            # Truncated normal with mean 0 and std_dev = 0.1   2HxC
            w = tf.Variable(tf.truncated_normal([2*hidden_size, no_of_output_labels+1], stddev = 0.1));
            # Zero bias initialization       C
            b = tf.Variable(tf.constant(0.1, shape = [no_of_output_labels+1]));
            # doing the affine projection BT1x2H * 2HxC + C -> BT1xC
            logits = tf.matmul(concat_out, w) + b;

            tf.summary.histogram('logits', logits);
        
            # reshaping back to original shape
            # BT1xC -> BxTxC
            logits = tf.reshape(logits, [self.batch_size, self.num_steps, (no_of_output_labels+1)]);
        
        
        
        #no need to use softmax activation because tensorflow CTC loss automatically performs softmax operation
        #setting up CTC
        #create CTC loss and decoder
        
        #first CTC requires an input of shape[TxBxC] (time major) , so perform a transpose
        # BxTxC -> TxBxC
        
        with tf.name_scope("Cost_calculation"):
            logits = tf.transpose(logits, [1, 0, 2]);
            self.gtTexts = tf.SparseTensor(
                tf.placeholder(tf.int64, shape = [None, 2]), #indices
                tf.placeholder(tf.int32,shape = [None]),     #Values
                tf.placeholder(tf.int64, shape = [2])        #shape
            )
            #calculate CTC loss for a batch
            self.seqLen = tf.placeholder(tf.int32, [None]) # dimention = batch_size
            loss = tf.nn.ctc_loss(
                labels = self.gtTexts,
                inputs = logits,
                sequence_length = self.seqLen,
                ctc_merge_repeated = True
            )

            #setup cost function for training

            self.cost = tf.reduce_mean(loss);
            tf.summary.scalar('cost', self.cost);
            
        with tf.name_scope("Decoder"):
            #setup decoder
            self.decoder = tf.nn.ctc_beam_search_decoder(inputs = logits, sequence_length = self.seqLen, beam_width = 80, merge_repeated = False)
        
        
        #setup gradient descent optimzer and train operation
        with tf.name_scope("Train"):
            self.batchesTrained = 0
            self.learning_rate = tf.placeholder(tf.float32, shape=[])
            self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.cost)
        
        
        #setup tensorflow saver and session
        print("Initializing tensorflow");
        print('Python: '+sys.version)
        print('Tensorflow: '+tf.__version__)
        
        with tf.name_scope("Save_model"):
            self.sess = tf.Session();

            self.saver = tf.train.Saver();
            model_dir = "model/";
            latest_snapshot = tf.train.latest_checkpoint(model_dir);

            self.merged = tf.summary.merge_all();

            if latest_snapshot:
                print("init with stored values from " + latest_snapshot);
                self.saver.restore(self.sess, latest_snapshot);
            else:
                print("init with new values");
                self.sess.run(tf.global_variables_initializer());
        
    
    
    def train_batch(self, batch_input, batch_output):
        #feed batch to nn to train it
        #recay learning rate
        rate = 0.01 if self.batchesTrained < 10 else (0.001 if self.batchesTrained < 10000 else 0.0001)
        (cost, _, summ) = self.sess.run([self.cost, self.train_op, self.merged],feed_dict = {self.inputs : batch_input,
                                                                            self.gtTexts : batch_output,
                                                                            self.seqLen : [self.num_steps] * self.batch_size,
                                                                            self.learning_rate : rate})
        self.batchesTrained += 1;
        return cost, summ;
    
    def validate_batch(self, batch_input):
        #feed input to rnn and apply decoder
        #initial state
        
        decoded = self.sess.run([self.decoder], feed_dict = {self.inputs : batch_input,
                                                        self.seqLen : [self.num_steps] * self.batch_size
                                                       });
        return self.to_text(decoded);

    def to_text(self, ctc_output):
        #extracts text output from ctc decoder output
        
        encodedStr = [[] for _ in range(self.batch_size)];
        
        decoded = ctc_output[0][0][0] #ctc output returns a tuple(decoded, neg_sum_logits) where decoded is a sparse tensor
        # go over all indices
        #print(decoded.dense_shape);
        for(idx, idx2d) in enumerate(decoded.indices):
            label = decoded.values[idx];
            
            batch_element = idx2d[0];
            time_step = idx2d[1];
            
            encodedStr[batch_element].append(label);
        
        return encodedStr;
    
    def save(self):
        #save model
        self.snap_id += 1;
        self.saver.save(self.sess, 'model/snapshot', global_step = self.snap_id);

Create evaluation function

In [4]:
def test_model(batch_size, input_test, output_test):
    m = Model(is_training = False, batch_size = batch_size, num_steps = 64, hidden_size =512, no_of_output_labels = 59, num_layers = 3);
    #evaluate
    train_ind = 0;
    correct_word = 0;
    errors = 0;
    total_char = 0;
    total_word = 0;
    input_word = input_test;
    output_label = output_test;
    recognized_sentence = "";
    while train_ind + batch_size <= len(input_word):
        iw = input_word[train_ind : train_ind + batch_size];
        ol = output_label[train_ind : train_ind + batch_size];

        input_data, output_data = iw, toSparse(ol);

        train_ind += batch_size;

        encodedString = m.validate_batch(input_data);
        #encodedString = m.validate_batch_r(sess, input_data);



        for _ in range(batch_size):
            #calculate error rate
            recog = to_string(encodedString[_], rs_borno);
            orig = to_string(ol[_], rs_borno);
            dist = editdistance.eval(recog, orig);

            recognized_sentence+=recog
            recognized_sentence+=" ";
            #print("recognized -> original");
            #print(recog," -> ", orig, "    edit distance is : ",dist);

            if(dist==0):correct_word+=1;
            else:
                errors += dist;

            total_char += len(orig);
            total_word += 1;

    error_rate = errors/total_char;
    word_accu = correct_word/total_word;

    #print("error rate is ",error_rate," and Word Accuracy is ", word_accu);
    return errors, recognized_sentence, error_rate, word_accu;

Iterate through sentences and evaluate them

In [5]:
error_style = list();
error_len = np.zeros(6);
ercnt = 0;

names = ["bappi", "bond", "delwar", "istiak", "sakib", "ruposh", "billal", "kamol", "maruf", "mazum", "nasim", "sayem", "sojib", "sumit", "tushar"];

#declare variables for evaluating different sentence lengths
word_accuracy_len = [list(), list(), list(), list(), list(), list()];
error_rate_len = [list(), list(), list(), list(), list(), list()];
sentence_accu_len = [list(), list(), list(), list(), list(), list()];
segment_error_len = [list(), list(), list(), list(), list(), list()];

#declare variables for evaluating different writing styles;
word_accuracy_style = list();
error_rate_style = list();
sentence_accu_style = list();
segment_error_style = list();

for name in names:
    #declare variables for evaluating single styles;
    word_accuracy_single_style = list();
    error_rate_single_style = list();
    sentence_accu_single_style = list();
    segment_error_single_style = list();
    for ind in range(1,(len(sentence)+1)):
        if(ind%3 != 2): continue;
        #input the image
        #if(ind==18): continue;
        lekha = cv2.imread('/media/s4k1b/Work/Study/Handwritting Recognition/BLSTM/sentence_image/'+name+'/'+str(ind)+'.png');
        #io.imshow(lekha);


        #preprocess the image
        grey = color.rgb2gray(lekha);
        threshold = filters.threshold_otsu(grey);
        binary = grey > threshold;
        inv = util.invert(binary);
        crop = ai.adjustImage(inv, True);
        #crop = mp.skeletonize(crop);
        crop = util.invert(crop);
        #io.imshow(crop);



        #separate to words
        pixelcnt = list();
        for j in range(crop.shape[1]):
            cnt = 0;
            for i in range(crop.shape[0]):
                if(crop[i][j] == False):
                    cnt = cnt + 1;
            pixelcnt.append(cnt);

        words = list();

        lastCrop = 0;
        index = 0;

        thresh = 12;


        while index < len(pixelcnt):
            #print("index = ",index);
            if(pixelcnt[index] == 0):
                zerocnt += 1;
            else:
                zerocnt = 0;

            if(zerocnt == thresh):
                #print(index-5+1);
                img = crop[:, lastCrop:index-thresh];
                words.append(resize2SquareKeepingAspectRation(ai.adjustImage(img, False), 64, cv2.INTER_AREA));
                zerocnt = 0;

                #find next collumn containing black pixel

                for nextindex in range(index, len(pixelcnt)):
                    if(pixelcnt[nextindex] != False):
                        #print("nextindex= ",nextindex);
                        lastCrop = nextindex;
                        index = nextindex - 1;
                        break;
            index+=1;

        if(lastCrop < index - 1):
            words.append(resize2SquareKeepingAspectRation(ai.adjustImage(crop[:, lastCrop:len(pixelcnt)-1], False), 64, cv2.INTER_AREA));


        #save the data
        print(ind);
        wrd = sentence[ind-1].split(" ");
        if(len(wrd) != len(words)):
            if(len(wrd)>len(words)):print("text sentence is greater");
            else: print("image sentence is greater");
            print("Detected different words");
            ercnt+=1;
            error_len[len(wrd)-2]+=1;
            segment_error_single_style.append(1);
            segment_error_len[len(wrd)].append(1);
        else:
            segment_error_single_style.append(0);
            segment_error_len[len(wrd)].append(0);
            input_word = list();
            output_label = list();
            for i in range(len(wrd)):
                greyi = color.rgb2gray(words[i]);
                thresholdi = filters.threshold_otsu(greyi);
                binaryi = greyi > thresholdi;
                #get feature vector
                features = gfv.get_features(binaryi);
                #append feature and output vectors
                input_word.append(features);
                output_label.append(glv.get_labels(wrd[i]));

            tf.reset_default_graph();
            errors, recog, error_rate, word_accuracy = test_model(len(output_label), input_word, output_label);
            print("Errors ", errors);
            print("Recognized", recog);

            if(errors == 0):
                sentence_accu_single_style.append(1);
                sentence_accu_len[len(wrd)].append(1);
            else:
                sentence_accu_single_style.append(0);
                sentence_accu_len[len(wrd)].append(0);

            word_accuracy_single_style.append(word_accuracy);
            error_rate_single_style.append(error_rate);
            
            word_accuracy_len[len(wrd)].append(word_accuracy);
            error_rate_len[len(wrd)].append(error_rate);
    
    
    #print(error_rate_single_style);
    sentence_accu_style.append(sentence_accu_single_style);
    word_accuracy_style.append(word_accuracy_single_style);
    error_rate_style.append(error_rate_single_style);
    segment_error_style.append(segment_error_single_style);
    
    
    
#evaluate for different writing style
print("\n\n FOR DIFFERENT WRITING STYLES\n\n");
print("Style \t Segmentation error \t Label error rate \t Word accuracy \t Sentence accuracy\n");
for (itr, name) in enumerate(names):
    sge = segment_error_style[itr].count(1)/len(segment_error_style[itr]);
    er = sum(error_rate_style[itr])/len(error_rate_style[itr]);
    wa = sum(word_accuracy_style[itr])/len(word_accuracy_style[itr]);
    sa = sentence_accu_style[itr].count(1)/len(sentence_accu_style[itr]);
    
    print(itr+1, " \t ", sge," \t ", er, " \t ", wa, " \t ", sa, "\n");
    
    
    
#evaluate for different sentence lengths
print("\n\n FOR DIFFERENT SENTENCE LENGTH\n\n");
print("Length \t Segmentation error \t Label error rate \t Word accuracy \t Sentence accuracy\n");
for itr in range(2,6):
    sge = segment_error_len[itr].count(1)/len(segment_error_len[itr]);
    er = sum(error_rate_len[itr])/len(error_rate_len[itr]);
    wa = sum(word_accuracy_len[itr])/len(word_accuracy_len[itr]);
    sa = sentence_accu_len[itr].count(1)/len(sentence_accu_len[itr]);
    
    print(itr, " \t ", sge," \t ", er, " \t ", wa, " \t ", sa, "\n");

2
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized আমি ভাত খাই 
5
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized তুমি কেমন আছো 
8
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized আমার নাম রহিম 
11
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized পুলিশকে ডাক 
14
Initiali

Errors  0
Recognized আমার নাম রহিম 
11
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized পুলিশকে ডাক 
14
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized বিদায় নিচ্ছি 
17
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized একটি ভাষা যথেষ্ট নয় 
20
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
E

Errors  0
Recognized একটি ভাষা যথেষ্ট নয় 
20
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized সে কোন দেশের লোক 
23
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized আপনি কি বাংলা বলতে পারেন 
26
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized মাফ করবেন 
29
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/

5
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized তুমি কেমন আছো 
8
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized আমার নাম রহিম 
11
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized পুলিশকে ডাক 
14
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized বিদায় নিচ্ছি 
17
Initia

Errors  3
Recognized আলটি ভাষা যথেষ্ট ময় 
20
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  6
Recognized সে কেমন তোােন লোক 
23
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized আপনি কি বাংলা বলতে পারেন 
26
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model/snapshot-18
Errors  0
Recognized মাফ করবেন 
29
Initializing tensorflow
Python: 3.6.5 (default, May 11 2018, 13:30:17) 
[GCC 7.3.0]
Tensorflow: 1.11.0
init with stored values from model/snapshot-18
INFO:tensorflow:Restoring parameters from model